## 分类任务微调全流程

首先配置镜像

In [1]:
import os
# 设置hf-mirror镜像地址
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# 查看是否成功结果
hf_endpoint = os.getenv('HF_ENDPOINT')
print('HF_ENDPOINT:', hf_endpoint)

HF_ENDPOINT: https://hf-mirror.com


构建hugging face数据集格式

In [2]:
from datasets import load_dataset
data_files = {"train":"./b站弹幕情感分析/data/train.tsv", "test":"./b站弹幕情感分析/data/dev.tsv"}
ChnSetiCorp_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
ChnSetiCorp_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text_a'],
        num_rows: 9146
    })
    test: Dataset({
        features: ['label', 'text_a'],
        num_rows: 1200
    })
})

加载模型评估metrics

In [3]:
import evaluate
metric = evaluate.combine(["accuracy", "f1", "precision", "recall", "matthews_correlation"])

模型标签对应

In [4]:
id2label = {0: '是恶评别看', 1: '会说话多说点'}
label2id = {'是恶评别看': 0, '会说话多说点': 1}

加载模型以及tokenizer

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# model_name = "yiyanghkust/finbert-tone-chinese"
model_name = "google-bert/bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, 
                                                           ignore_mismatched_sizes=True,
                                                           id2label = id2label,
                                                           label2id = label2id) 
#num_labels是输出的类别数量

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

整个数据集的tokenizer

In [7]:
def preprocess_function(example):
    return tokenizer(example["text_a"], truncation=True, max_length=128)

In [8]:
encoded_dataset = ChnSetiCorp_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [9]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text_a', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9146
    })
    test: Dataset({
        features: ['label', 'text_a', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
})

参数初始化

In [10]:
from transformers import TrainingArguments, Trainer
# model_name = "wosuibianzhaode"
task = "sentiment"
batch_size = 32
metric_name = "matthews_correlation"
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    report_to="none"
    # push_to_hub=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


计算metrics的函数

In [11]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

训练

In [12]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_9484/1443369649.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Matthews Correlation
1,No log,0.214385,0.921667,0.917976,0.951175,0.887015,0.845074
2,0.216200,0.200186,0.935000,0.934564,0.929883,0.939292,0.870041
3,0.216200,0.274796,0.931667,0.928322,0.963702,0.895447,0.865330


模型评估（会根据效果最好的模型结果展示评估结果）

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3044030964374542,
 'eval_accuracy': 0.9433333333333334,
 'eval_f1': 0.9420783645655877,
 'eval_precision': 0.9518072289156626,
 'eval_recall': 0.9325463743676222,
 'eval_matthews_correlation': 0.886802290168004,
 'eval_runtime': 2.0055,
 'eval_samples_per_second': 598.341,
 'eval_steps_per_second': 18.947,
 'epoch': 5.0}

使用训练好的模型做推理

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model = "/home/pod/shared-nvme/NLP-study/文本分类/google-bert/bert-base-chinese-finetuned-sentiment/checkpoint-1430", device=0)
classifier

In [ ]:
text = "李洋是个臭傻逼"
classifier(text)

[{'label': 'LABEL_0', 'score': 0.9968191385269165}]